In [95]:
import pandas as pd
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, WheelZoomTool)
from bokeh.models.widgets import Select
from bokeh.layouts import column, row
from bokeh.palettes import PuOr11
from bokeh.plotting import figure, show, save, output_file
import json
import geopandas as gpd

In [96]:
shp = gpd.read_file('nycshapes.json')
shp = shp.to_crs("EPSG:2263")

In [97]:
data = pd.read_csv('allheadtoheads.csv')

In [98]:
shp = shp.merge(data, left_on='elect_dist', right_on='Precinct')
geosource = GeoJSONDataSource(geojson = shp.to_json())

In [99]:
candidateToNum = {"Adrienne E. Adams": 1,
                 "Andrew M. Cuomo": 2,
                 "Brad Lander": 3,
                 "Jessica Ramos": 4,
                 "Michael Blake": 5,
                 "Paperboy Love Prince": 6,
                 "Scott M. Stringer": 7,
                 "Selma K. Bartholomew": 8,
                 "Whitney R. Tilson": 9,
                 "Zellnor Myrie": 10,
                 "Zohran Kwame Mamdani": 11}
candidateTuples = []
for i in candidateToNum:
    candidateTuples.append((candidateToNum[i], i))

In [100]:
candidateTuples

[(1, 'Adrienne E. Adams'),
 (2, 'Andrew M. Cuomo'),
 (3, 'Brad Lander'),
 (4, 'Jessica Ramos'),
 (5, 'Michael Blake'),
 (6, 'Paperboy Love Prince'),
 (7, 'Scott M. Stringer'),
 (8, 'Selma K. Bartholomew'),
 (9, 'Whitney R. Tilson'),
 (10, 'Zellnor Myrie'),
 (11, 'Zohran Kwame Mamdani')]

In [101]:
p = figure(title = "Results",
           sizing_mode = "stretch_both",
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, reset')
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_size = "0pt"
p.yaxis.major_label_text_font_size = "0pt"
p.xgrid.grid_line_color = None
p.toolbar.active_drag = None
p.ygrid.grid_line_color = None
dropdownCand1 = Select(title = "Candidate 1", value = 11, options = candidateTuples)
dropdownCand2 = Select(title = "Candidate 2", value = 2, options = candidateTuples)
colorMapper = LinearColorMapper(palette = PuOr11, low = -1, high = 1)

precincts = p.patches('xs','ys', source = geosource,
                   fill_color = {'field': f"11-2", 'transform': colorMapper},
                   line_color = 'white', 
                   line_width = 0.05, 
                   fill_alpha = 1)
hover = HoverTool(
    tooltips=[
        ('Precinct', f'@{{{f"elect_dist"}}}'),
        ('Votes for Candidate 1', f'@{{{f"11 over 2"}}}{{0,0}}'),
        ('Votes for Candidate 2', f'@{{{f"2 over 11"}}}{{0,0}}')]
)
p.add_tools(hover)
callback = CustomJS(args=dict(
    source = geosource,
    dropdownCand1 = dropdownCand1,
    dropdownCand2 = dropdownCand2,
    precincts = precincts,
    plot = p,
    candidateToNum = candidateToNum,
    colorMapper = colorMapper,
    hover = hover
), code = """
    const cand1 = dropdownCand1.value;
    const cand2 = dropdownCand2.value;
    const votes1Column = `${cand1} over ${cand2}`;
    const votes2Column = `${cand2} over ${cand1}`;
    var columnName = `${cand1}-${cand2}`;
    const colorData = source.data[columnName];
    precincts.glyph.fill_color = {field: columnName, transform: colorMapper};
    hover.tooltips = [
        ['Precinct', '@{' + `elect_dist` + '}']
        ['Votes for Candidate 1', '@{' + votes1Column + '}'],
        ['Votes for Candidate 2', '@{' + votes2Column + '}']
    ];
""")

dropdownCand1.js_on_change('value', callback)
dropdownCand2.js_on_change('value', callback)

controls = row(dropdownCand1, dropdownCand2)
layout = column(controls, p, sizing_mode="stretch_both", aspect_ratio=1.25)
show(layout)

In [102]:
output_file("index.html", mode='cdn')
save(layout)

'c:\\Users\\austi\\Downloads\\2025_Primary_CVR_2025-07-17\\index.html'